In [1]:
%matplotlib inline
from IPython.core.pylabtools import figsize

from functools import partial
import itertools
import re

import matplotlib.gridspec as gridspec
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import pylab as pl
import scipy as sp
from graphviz import Digraph
import pydot
from IPython.display import Image
import pandas as pd
import seaborn as sns

print(__doc__)
import pandas as pd
from patsy import dmatrices
import scipy as sc
import scipy.optimize as opt
import statsmodels.api as sm

import sklearn as sk


from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import TransformerMixin
from sklearn.pipeline import BaseEstimator
from sklearn.base import BaseEstimator, TransformerMixin, clone
from sklearn.utils.metaestimators import if_delegate_has_method
from sklearn.utils import column_or_1d
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import make_union


from sklearn.datasets import samples_generator
from sklearn import datasets

from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import FunctionTransformer

from sklearn.feature_extraction import DictVectorizer

import sklearn.feature_extraction as fe

import sklearn.feature_selection as fs
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import VarianceThreshold

import sklearn.linear_model as lm
import sklearn.grid_search as gs
from sklearn.svm import SVC
from sklearn.decomposition import PCA


import sklearn.preprocessing as preprocessing
import sklearn.cross_validation as cv
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_curve, auc

from sklearn_pandas import DataFrameMapper, cross_val_score

import matplotlib.pyplot as plt
import patsy
from statsmodels.sandbox import cox

from patsylearn import *

import joblib as jb


from datasu.auc import *
from datasu.dicts import *
from datasu.files import *
from datasu.pandas import *
from datasu.persist import *
from datasu.patsy import *

Automatically created module for IPython interactive environment


/root/.pyenv/versions/2.7.9/envs/2.7.9-analytics2/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


## UTILS

## LOAD DATA

In [2]:
main_folder = '/ea/Datasets/Kaggle-AcquireValuedShoppersChallenge2/'
df_coupons = pd.read_csv(main_folder+'offers')[['offer','category','company','brand','offervalue','quantity']]
df_offers_ids = pd.read_csv(main_folder+'trainHistory').rename(columns={'id': 'customer_id'})
df_offers_ids_submission = pd.read_csv(main_folder+'testHistory').rename(columns={'id': 'customer_id'})
# df_trans_all = pd.read_csv(main_folder+'transactions_reduced_category').rename(columns={'id': 'customer_id'})

In [3]:
df_offers_all = pd.merge(df_offers_ids, df_coupons, on=['offer'])
df_offers_all = df_offers_all[['customer_id','chain','offer','market','category','company','brand','offerdate','offervalue','quantity','repeattrips','repeater']]

df_offers_all_submission = pd.merge(df_offers_ids_submission, df_coupons, on=['offer'])
df_offers_all_submission = df_offers_all_submission[['customer_id','chain','offer','market','category','company','brand','offerdate','offervalue','quantity']]

In [4]:
df_coupons.head()

,offer,category,company,brand,offervalue,quantity
0,1190530,9115,108500080,93904,5.00,1
1,1194044,9909,107127979,6732,1.00,1
2,1197502,3203,106414464,13474,0.75,1
3,1198271,5558,107120272,5072,1.50,1
4,1198272,5558,107120272,5072,1.50,1


In [5]:
df_coupons[df_coupons.offer==1213242]

,offer,category,company,brand,offervalue,quantity
26,1213242,5824,105190050,26456,2,1


In [6]:
df_offers_all.head()

,customer_id,chain,offer,market,category,company,brand,offerdate,offervalue,quantity,repeattrips,repeater
0,86246,205,1208251,34,2202,104460040,3718,2013-04-24,2,1,5,t
1,15753725,17,1208251,4,2202,104460040,3718,2013-04-24,2,1,0,f
2,16535563,4,1208251,1,2202,104460040,3718,2013-04-27,2,1,7,t
3,18259179,3,1208251,2,2202,104460040,3718,2013-04-24,2,1,0,f
4,21024070,15,1208251,9,2202,104460040,3718,2013-04-23,2,1,1,t


In [7]:
df_trans_all.head()

NameError: name 'df_trans_all' is not defined

In [ ]:
offer_counts = df_offers_all.groupby(by='offer').count().customer_id.order(ascending=False)
sns.barplot(y=offer_counts.index.values, x=offer_counts.values, orient='h')

## FILTER DATA

In [8]:
df_trans = df_trans_all

NameError: name 'df_trans_all' is not defined

In [9]:
df_offers_filtered = df_offers_all#[df_offers_all.offer==1197502]

In [10]:
len(df_trans.customer_id.unique()), len(df_trans.customer_id),  len(df_offers_filtered.customer_id.unique()),len(df_offers_filtered.customer_id), 

NameError: name 'df_trans' is not defined

## FEATURE EXTRACTION

### AGGREGATE TRANSCATIONS

In [11]:
load_variables(path=main_folder+'working_data');

['df_trans_grp_customer_brand.var', 'df_trans_grp_customer_category.var', 'df_trans_grp_customer_dept.var']
loaded df_trans_grp_customer_brand
loaded df_trans_grp_customer_category
loaded df_trans_grp_customer_dept


### PERSIST DATA TO DISK

### MERGE FEATURES TO ONE TABLE

In [12]:
print len(df_offers_all)
df_offers_all.head()

160057


,customer_id,chain,offer,market,category,company,brand,offerdate,offervalue,quantity,repeattrips,repeater
0,86246,205,1208251,34,2202,104460040,3718,2013-04-24,2,1,5,t
1,15753725,17,1208251,4,2202,104460040,3718,2013-04-24,2,1,0,f
2,16535563,4,1208251,1,2202,104460040,3718,2013-04-27,2,1,7,t
3,18259179,3,1208251,2,2202,104460040,3718,2013-04-24,2,1,0,f
4,21024070,15,1208251,9,2202,104460040,3718,2013-04-23,2,1,1,t


In [13]:
df_offers_all_merge4 = merge_data(df_offers_all, how='left',
                                  rights=[(df_trans_grp_customer_brand,['customer_id','brand']),
                                          (df_trans_grp_customer_category,['customer_id', 'category']),
#                                           (df_trans_grp_customer_dept,['customer_id'])
                                         ])

df_offers_all_merge4.fillna(0, inplace=True)

160057
160057
160057


In [14]:
len(df_offers_all), len(df_offers_all_merge4), len(df_offers_all_merge4.columns)

(160057, 160057, 26)

In [15]:
df_offers_all_merge4.brand.unique()

array([  3718,  13474, 102504,   7668,  15889,    875,  13791,   5072,
         6926,  64486,  28840,   6732])

### CLEAR MEMORY

In [16]:
# %reset_selective -f df_trans_all
# %reset_selective -f df_trans

## TRAIN MODEL

In [17]:
df_data_train = df_offers_all_merge4

In [18]:
df_data_train.columns

Index([u'customer_id', u'chain', u'offer', u'market', u'category', u'company',
       u'brand', u'offerdate', u'offervalue', u'quantity', u'repeattrips',
       u'repeater', u'agg_productsize_customer_id_brand_average',
       u'agg_productsize_customer_id_brand_total',
       u'agg_customer_id_customer_id_brand_count',
       u'agg_purchaseamount_customer_id_brand_average',
       u'agg_purchaseamount_customer_id_brand_total',
       u'agg_purchasequantity_customer_id_brand_average',
       u'agg_purchasequantity_customer_id_brand_total',
       u'agg_productsize_customer_id_category_total',
       u'agg_productsize_customer_id_category_average',
       u'agg_customer_id_customer_id_category_count',
       u'agg_purchaseamount_customer_id_category_total',
       u'agg_purchaseamount_customer_id_category_average',
       u'agg_purchasequantity_customer_id_category_total',
       u'agg_purchasequantity_customer_id_category_average'],
      dtype='object')

In [19]:
df_data_train.repeattrips.sum()

105051

In [20]:
categorical_cols = ['chain','market','category','company','brand']
agg_cols = filter(lambda c: re.match("agg_*",c), df_data_train.columns)
num_cols = ['offervalue','quantity']


In [21]:
df_X = df_data_train.filter(items=categorical_cols+agg_cols+num_cols) 
columns_to_type(df_X, categorical_cols, 'string')
columns_to_type(df_X, agg_cols+num_cols, 'double')

df_Y = df_data_train[['repeater']]
Y = df_Y.repeater.map(lambda v: v=='t')
W = df_data_train.repeattrips.replace(to_replace=0, value=1).values

In [22]:
categorical_cols

['chain', 'market', 'category', 'company', 'brand']

In [23]:
df_X.columns

Index([u'chain', u'market', u'category', u'company', u'brand',
       u'agg_productsize_customer_id_brand_average',
       u'agg_productsize_customer_id_brand_total',
       u'agg_customer_id_customer_id_brand_count',
       u'agg_purchaseamount_customer_id_brand_average',
       u'agg_purchaseamount_customer_id_brand_total',
       u'agg_purchasequantity_customer_id_brand_average',
       u'agg_purchasequantity_customer_id_brand_total',
       u'agg_productsize_customer_id_category_total',
       u'agg_productsize_customer_id_category_average',
       u'agg_customer_id_customer_id_category_count',
       u'agg_purchaseamount_customer_id_category_total',
       u'agg_purchaseamount_customer_id_category_average',
       u'agg_purchasequantity_customer_id_category_total',
       u'agg_purchasequantity_customer_id_category_average', u'offervalue',
       u'quantity'],
      dtype='object')

In [24]:
print len(categorical_cols+agg_cols), len(df_X.columns)
print len(df_X),len(df_Y)

19 21
160057 160057


### BUILD INTERACTIONS FEATURES

In [26]:
# col_name_pattern = lambda prefix,infix: "%s.*%s.*" % (prefix,infix)

# filter_array = lambda pattern, array: filter(lambda c: re.match(pattern,c), array)
def get_expr(members, operator, expr='%s',pref='', suf=''):
    if not isinstance(members,list):
        members = [members]
    members = map(lambda m:expr % m ,members)
    return ''.join([pref, operator.join(members), suf]) if len(members) > 0 else ''

# agg_col_name_pattern = partial(col_name_pattern, 'agg')
plus_expr = partial(get_expr, operator='+')
plus_categorical_expr = partial(plus_expr, expr='C(%s)')
# plus_agg_columns_by_infix1 = lambda infix: plus_expr(filter_columns(agg_col_name_pattern(infix)))
plus_agg_columns_by_infix = lambda infix, columns: get_expr(filter_array(col_name_pattern('agg',infix), columns),'+')


In [27]:
categorical_cols_plus = plus_categorical_expr(categorical_cols)

brand_interactions = "(%s):(%s)" % (plus_agg_columns_by_infix('brand',agg_cols), categorical_cols_plus)
category_interactions = "(%s):(%s)" % (plus_agg_columns_by_infix('category',agg_cols), categorical_cols_plus)
num_interactions = "(%s):(%s)" % (plus_expr(agg_cols), plus_expr(num_cols))

patsy_expr = plus_expr([brand_interactions,category_interactions,num_interactions])
patsy_expr

'(agg_productsize_customer_id_brand_average+agg_productsize_customer_id_brand_total+agg_customer_id_customer_id_brand_count+agg_purchaseamount_customer_id_brand_average+agg_purchaseamount_customer_id_brand_total+agg_purchasequantity_customer_id_brand_average+agg_purchasequantity_customer_id_brand_total):(C(chain)+C(market)+C(category)+C(company)+C(brand))+(agg_productsize_customer_id_category_total+agg_productsize_customer_id_category_average+agg_customer_id_customer_id_category_count+agg_purchaseamount_customer_id_category_total+agg_purchaseamount_customer_id_category_average+agg_purchasequantity_customer_id_category_total+agg_purchasequantity_customer_id_category_average):(C(chain)+C(market)+C(category)+C(company)+C(brand))+(agg_productsize_customer_id_brand_average+agg_productsize_customer_id_brand_total+agg_customer_id_customer_id_brand_count+agg_purchaseamount_customer_id_brand_average+agg_purchaseamount_customer_id_brand_total+agg_purchasequantity_customer_id_brand_average+agg_pu

### BUILD PIPELINE

In [ ]:
patsy_transf = make_pipeline(
    PatsyTransformer(patsy_expr, return_type='dataframe'))

In [ ]:
# mapper = DataFrameMapper([
#         (col, sk.preprocessing.LabelBinarizer()) for col in categorical_cols] 
#       + [(agg_cols+num_cols, None)])

prepare_pipeline = make_pipeline(patsy_transf,
                        PolynomialFeatures(degree=1, interaction_only=True),                         
                        preprocessing.StandardScaler()
                        ) 

prepare_pipeline2 = make_pipeline(
                        PolynomialFeatures(degree=1, interaction_only=True),                         
                        preprocessing.StandardScaler()
                        ) 

In [ ]:
fit_pipeline_LR1 = make_pipeline(prepare_pipeline,
                             lm.LogisticRegression(C=0.01, penalty='l1'))     

fit_pipeline2_LR1 = make_pipeline(prepare_pipeline2,
                             lm.LogisticRegression(C=0.01, penalty='l1'))     

fit_pipeline3_LR1 = make_pipeline(prepare_pipeline2,
                             lm.LogisticRegression(C=0.001, penalty='l1'))     
# class_weight=W_train


In [ ]:
fit_pipeline_SGD1 = make_pipeline(prepare_pipeline,
                                 lm.SGDClassifier(loss="hinge", penalty="l1", alpha=0.01))

In [ ]:
fit_pipeline_SVC1 = make_pipeline(prepare_pipeline,
                                 SVC())

### SPLIT TRAIN-TEST

In [ ]:
df_X_train, df_X_test, Y_train, Y_test, W_train, W_test = train_test_split(df_X, Y, W)
len(df_X_train),len(df_X_test)


### FIT MODEL

In [ ]:
df_X_transf = prepare_pipeline.fit_transform(df_X_train)
len(df_X_transf[1]),len(df_X_transf), len(Y)

In [ ]:
df_X_transf.shape

In [ ]:
fit_pipeline_LR1.fit(df_X_train, y=Y_train)
print 'score in-sample: ' + str(fit_pipeline_LR1.score(df_X_train, y=Y_train))

In [ ]:
logreg = fit_pipeline_LR1.named_steps['logisticregression']
logreg.classes_
sum(logreg.coef_[0,:])

In [ ]:
probas_LR1 = fit_pipeline_LR1.decision_function(df_X_test)
plot_auc(Y_test, probas_LR1)

### CROSSVALIDATION

In [ ]:
scores = cv.cross_val_score(fit_pipeline_LR1, df_X_train, Y_train,cv=5, scoring='accuracy')
print scores
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

# X_test = prepare_pipeline.transform(X_test)
# print 'Test accuracy: %0.2f' % fit_pipeline.score(X_test, Y_test


In [ ]:
run_crossval(df_X_train, Y_train, fit_pipeline_LR1)

In [ ]:
W_train

### TEST

In [ ]:
model_SGD1 = fit_pipeline_SGD1.fit(df_X_train, y=Y_train)
model_LR1 = fit_pipeline_LR1.fit(df_X_train, y=Y_train)
model_SVC1 = fit_pipeline_SVC1.fit(df_X_train, y=Y_train)

In [ ]:
probas_LR1 = model_LR1.decision_function(df_X_test)
plot_auc(Y_test, probas_LR1)
 
probas_SGD1 = model_SGD1.decision_function(df_X_test)
plot_auc(Y_test, probas_SGD1)

probas_SVC1 = model_SVC1.decision_function(df_X_test)
plot_auc(Y_test, probas_SVC1)

## TEST ON KAGGLE

### PREPARA DATA FOR PREDICTION

In [ ]:
df_offers_all_submission.head()

In [ ]:
df_offers_all_submission_merge4 = merge_data(df_offers_all_submission, how='left',
                                             rights=[(df_trans_grp_customer_brand,['customer_id','brand']),
                                                     (df_trans_grp_customer_category,['customer_id', 'category']),
#                                                      (df_trans_grp_customer_dept,['customer_id'])
                                                    ])
df_offers_all_submission_merge4.fillna(0, inplace=True)

In [ ]:
len(df_offers_all_submission_merge4), len(df_offers_all_submission.customer_id.unique())

In [ ]:
df_offers_all_submission_merge4[df_offers_all_submission_merge4.customer_id==12262064]

In [ ]:
df_X_submission = df_offers_all_submission_merge4.filter(items=categorical_cols+agg_cols+num_cols) 
columns_to_type(df_X_submission, categorical_cols, 'string')
columns_to_type(df_X_submission, agg_cols+num_cols, 'double')


### FIT MODEL ON ALL DATA

In [ ]:
data_chunked = [df_X, df_X_submission]
dinfo = patsy.incr_dbuilder(patsy_expr, lambda: iter(data_chunked))



In [ ]:
df_X_tranf = patsy.dmatrix(dinfo.design_info,df_X_train, return_type='dataframe')
df_X_submission_transf = patsy.dmatrix(dinfo.design_info,df_X_submission, return_type='dataframe')

In [ ]:
len(df_X_tranf.columns)

In [ ]:
fit_pipeline3_LR1.fit(df_X_tranf, y=Y_train)
print 'score in-sample: ' + str(fit_pipeline3_LR1.score(df_X_tranf, y=Y_train))

In [ ]:
df_X_test_tranf = patsy.dmatrix(dinfo.design_info,df_X_test, return_type='dataframe')
probas_LR1 = fit_pipeline3_LR1.decision_function(df_X_test_tranf)
plot_auc(Y_test, probas_LR1)

## PREDICT SUBMISSION

In [ ]:
probas_ = fit_pipeline3_LR1.decision_function(df_X_submission_transf).astype(float)

In [ ]:
len(probas_)


In [ ]:
sum(probas_),sum(probas_)

In [ ]:
df_submission = df_offers_all_submission.customer_id.to_frame(name='id')
df_submission['repeatProbability'] = probas_.tolist()
df_submission.head()

In [ ]:
df_submission.to_csv(path_or_buf=main_folder+'submission12_LR1_interactions', index=False)

## OTHER

In [ ]:
data_chunked = [df_X, df_X_submission]
dinfo = patsy.incr_dbuilder(patsy_expr, lambda: iter(data_chunked))

bd = patsy.dmatrix(dinfo.design_info,df_X_submission, return_type='dataframe')

In [ ]:
len(bd)

In [ ]:
X = patsy.dmatrix('market + category -1', data=df_off_hist_train, return_type='dataframe')
X.head()

In [ ]:
pd.DataFrame(df_trans_grp_brand.to_records())

In [ ]:
X = patsy.dmatrix('total_int_calls_minutes:C(intl_package, Treatment) + age -1', data=X_train, return_type='dataframe')

In [ ]:
df_offers_history[df_offers_history.id==4638754711]

In [ ]:
df_off_hist_train['id_chain'] =  df_off_hist_train['id'].map(lambda x:(str(x)+'_')) + df_off_hist_train['chain'].map(str)

# PREPARE FILES

In [ ]:
from datetime import datetime
import zipfile

loc_history = "./testHistory"

loc_transactions = "./transactions"
loc_reduced = "./transactions_reduced_customer_test" # will be created




In [ ]:
reduce_data(loc_history, loc_transactions, loc_reduced)
